In [ ]:
#We define initial parameters
%matplotlib inline

region_file="observed_target_info.reg"

output_file="output FUV.csv"

fits_directory="." #The directory to look for .fits files in   

file_key="fd-int" #a string located in all the files you want to analyze in fits_directory

flux_conv = 2.06*(10**(-16)) 
        # A conversion factor from pixel count to flux in units of (erg cm-2 Å-1)/(N/s)
        #NUV: Flux = 2.06*10**(-16) x CPS, FUV: Flux = 1.40*10**(-15) x CPS
        #Reference: http://asd.gsfc.nasa.gov/archive/galex/FAQ/counts_background.html

flux_conv_err = 0 #error in flux_conv
    
pixel_area=(1.5/60)**2 #Arcmin^2 per pixel - given in arcseconds, converted to arcmin, and squared to find area in arcmin^2.

pixel_area_err = 0 #error in pixel_area

Pec_error = ((1+300/(299792.458))/(1-300/(299792.458)))**(1/2)-1  #The error in redshift due to peculiar velocity


In [ ]:
#General use
import numpy as np
from astropy.io import fits
from astropy.cosmology import WMAP9 as cosmo

#For creating a list of .fits files to perform photometry on
import os

#For reading and creating tables
from astropy.table import Table, Column
import astropy.io.ascii as asc  

#For performing photometry
from astropy import units as u
from photutils import aperture_photometry, SkyCircularAperture, CircularAperture
from astropy.coordinates import SkyCoord
from astropy.wcs import WCS

#For showing .fits images
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm


In [ ]:
#We define functions used to calculate error

def Conv_error(val,err): #uncertanty in the conversion factor from arcmin^2 to Kpc^2 using arguments (redshift, redshift error)
    return(
            abs(((1/cosmo.kpc_comoving_per_arcmin(val+err))**2-(1/cosmo.kpc_comoving_per_arcmin(val-err))**2)/2).value
          )

def Comov_error(val,err): #Uncertanty in the comoving_distance using arguments (redshift, redshift error)
    return(
            abs(((cosmo.comoving_distance(val+err).cgs)-(cosmo.comoving_distance(val-err).cgs))/2).value          
                )

def Flux_error(val_1,err_1): #uncertanty in flux using arguments (counts per second, error in counts per second)
    return(
        ((val_1*flux_conv_err)**2+(flux_conv*err_1)**2)**(1/2)
            )

def Luminosity_error(flux,flux_err,dist,dist_err): #The input 'dist' is Comoving Distance
    return(
        ((4*np.pi*(dist**2)*flux_err)**2+(8*np.pi*dist*flux*dist_err)**2)**(1/2)
    )

def Sbrightness_error(lum,lum_err,conv,conv_err): #Uncertanty in surface brightness
    return(
        ((conv*lum_err/pixel_area)**2+(lum*conv_err/pixel_area)**2+
         (lum*conv*pixel_area_err/(pixel_area)**2)**2)**(1/2)
    )


In [ ]:
#When we perfom photometry on a .fits file, photutils may return zero. This can be because the pixel's in the aperture
#are not within the field of view (towards the edge of the fits file) or that they are within the field of view
#but the observed flux is actually zero. In such a case this function may be called to display the .fits image along
#with the aperture so that the user can confirm which case it is.

def zero_check(sci,aperture,wcs,w):

    plt.imshow(sci, cmap='gray', norm=LogNorm())
    aperture.to_pixel(wcs).plot(color='red')
    CircularAperture(w,100).plot(color='red')
    plt.show()
    
    
        
    if 'n' in input(): return(False)
    else: return(True)

In [ ]:
#We define a function to perform photometry on a .fits file. When given the file path of a .fits file it checks to see
#if there is a supernova observed in that file. If it finds one it returns the supernova name, a table of photometry 
#results, and the .fits file's exposure time

def photometry(fits_file):
    #open the file and create an hdulist
    hdulist = fits.open(fits_file)
    sci = hdulist[0].data
    wcs = WCS(fits_file)
    print('\n',fits_file)

    for sn in cord:

        #Define the SN location in pixels
        w = wcs.all_world2pix(cord[sn].ra,cord[sn].dec, 1)
        
        z=100
        
        #Make sure the sn is located in the image, and near the data pixels (aka 200 pixels away from the image edge)
        if 190<w[0]<3600 and 190<w[1]<3600 and True in (sci[int(w[1])-z:int(w[1])+z,int(w[0])-z:int(w[0])+z]!=0):

            #get exposure time and calculate error array
            exp_time = hdulist[0].header['EXPTIME']
            error = ((hdulist[0].data*exp_time)**(1/2))/exp_time
            
            #Find arcmin of 1kpc radius region
            r = 2*u.kpc/cosmo.kpc_comoving_per_arcmin(float(red[sn]))
            
            #Create an aperture
            aperture = SkyCircularAperture(cord[sn], r) 
            

            #Perfom photometry
            phot_table = aperture_photometry(hdulist[0], aperture, error=error)
            

            #We add values to their respective lists, which will be writen to the output file. 
            #Note that if the photometry = 0, we run zero_check which lets the user manually determine if 
            #the aperture is in the data region
            if phot_table[0][0]!=0 or (phot_table[0][0]==0 and zero_check(sci,aperture,wcs,w)):
                return([sn,phot_table,hdulist[0].header['EXPTIME']])
            
    #close the hdulist
    hdulist.close()
    return(False)


In [ ]:
#We create a list of .fits files to perform photometry on

file_list = []

for path, subdirs, files in os.walk(fits_directory):
    for name in files:
        if file_key in name: file_list.append(os.path.join(path, name))

In [ ]:
#Create a dictionary of coordinates (in degrees) and redshift by using values from the .reg file

reg = asc.read(region_file, data_start=2, delimiter = "#", header_start=2)
cord, red = {}, {}

for row in reg: 
    cord[row[1].split(",")[0].strip('text={}')] = SkyCoord(row[0].strip('point()').replace(',',' '), unit=(u.hourangle, u.deg))
    red[row[1].split(",")[0].strip('text={}')] = row[1].split(",")[2].strip('}').replace('z=','')


In [ ]:
#We perform photometry on each .fits file

name, redshift, photom, photom_err, expt, file_name = [], [], [], [], [], [] #lists to be used when creating the output file

for fits_file in file_list:
    p = photometry(fits_file)
    if p!= False        print(p[0],':', p[1][0][0], '+/-', p[1][0][1])
        
        name.append(p[0])
        redshift.append(float(red[p[0]]))
        photom.append(p[1][0][0])
        photom_err.append(p[1][0][1])
        expt.append(p[2])
        file_name.append(fits_file)
        
    elif p==False: print('No Supernova Found')

    
print('\nPhotometry finished')


In [ ]:
#We calculate the remaining values in the table

out = Table()
out.keep_columns([])
        
out['Sn Name'] = name

out['Red shift'] = redshift

out['Redshift error'] = [((1/1000)**2+(Pec_error)**2)**(1/2) for row in out] 
#Error in redshift taken as 1 in 1000 with an additional 300 km/s contribution from peculiar velocity

out['ArcMin^2 per Kpc^2 at Redshift'] =[
    (1/cosmo.kpc_comoving_per_arcmin(row[1]).value)**2 for row in out] 

out['ArcMin^2 / Kpc^2 error'] = [Conv_error(row[1],row[2]) for row in out] 

out['Photometry'] = photom

out['Exposure Time'] = expt

out['Photometry Error N^(1/2)/s'] = photom_err

out['Flux (erg s-1 cm-2 A-1 px-1)'] = [flux_conv*row[5] for row in out] #convert cps to flux using the conversion factor

out['Flux error (erg s-1 cm-2 A-1 px-1)'] = [Flux_error(row[5],row[7]) for row in out] 

out['Comoving Distance at Redshift (cm)'] = [cosmo.comoving_distance(row[1]).cgs.value for row in out] 

out['Comoving Error (cm)'] = [Comov_error(row[1],row[2]) for row in out]

out['Luminosity (erg s-1 A-1 px-1)'] = [row[8]*4*np.pi*row[10]**2 for row in out] #luminosity = flux*4*pi*r^2

out['Luminosity Error (erg s-1 A-1 px-1)'] = [Luminosity_error(row[8],row[9],row[10],row[11]) for row in out]

out['Surface Brightness (erg s-1 A-1 Kpc^-2)'] = [row[12]/pixel_area*row[3] for row in out]

out['Surface Brightness error (erg s-1 A-1 Kpc^-2)'] = [Sbrightness_error(row[12],row[13],row[3],row[4]) for row in out]

out['log10 of Surface Brightness'] = [np.log10(row[14]) for row in out]

out['Error in log10'] = [row[15]/(row[14]*np.log(10)) for row in out]

out['File Name'] = file_name

out.remove_columns(['Comoving Distance at Redshift (cm)', 'Comoving Error (cm)']) #remove columns we don't want in the output file

In [ ]:
out

In [ ]:
#Write data to an output file specified by the output_file object

asc.write(out, output_file, delimiter=",")
print('Script finished')
